In [ ]:
ENV_TYPE = "TEST1"

if(ENV_TYPE != "TEST"):
  !git clone "https://github.com/kk-digital/kcg-ml-sd1p4.git"
  %cd kcg-ml-sd1p4
  !pip3 install -r requirements.txt
  exit()
else:
  !pip3 install -r requirements.txt --ignore-installed

In [ ]:
!python3 ./download_models.py

In [ ]:
!python3 ./process_models.py

In [ ]:
import os, sys
if(os.getcwd() == "/content"):
  %cd kcg-ml-sd1p4
import torch
from typing import List
import hashlib
import json
import math
import numpy as np
import safetensors as st
from os.path import join

from chad_score.chad_score import ChadScorePredictor
from stable_diffusion import StableDiffusion
from stable_diffusion.model.clip_text_embedder import CLIPTextEmbedder
from stable_diffusion.model.clip_image_encoder import CLIPImageEncoder
from stable_diffusion.utils_model import *
from stable_diffusion.utils_backend import *
from stable_diffusion.utils_model import initialize_latent_diffusion
from stable_diffusion.utils_image import *
from stable_diffusion.constants import IODirectoryTree
from stable_diffusion.constants import TOKENIZER_PATH, TEXT_MODEL_PATH
from transformers import CLIPTextModel, CLIPTokenizer


base_dir = "./"
sys.path.insert(0, base_dir)

# Variables
SEED = 0
BATCH_SIZE = 1
POPULATION_SIZE = 12
GEN_IMAGE_N_GENERATIONS = 50
CFG_STRENGTH = 9
N_STEPS = 20
EMBEDDED_PROMPTS_DIR = os.path.abspath(join(base_dir, "./input/embedded_prompts/"))
OUTPUT_DIR = os.path.abspath(
    os.path.join(base_dir, "./output/ga/")
)
IMAGES_DIR = os.path.abspath(join(OUTPUT_DIR, "images/"))
FEATURES_DIR = os.path.abspath(join(OUTPUT_DIR, "features/"))

NULL_PROMPT = ""
# DEVICE = input("Set device: 'cuda:i' or 'cpu'")
DEVICE = get_device()

print(EMBEDDED_PROMPTS_DIR)
print(OUTPUT_DIR)
print(IMAGES_DIR)
print(FEATURES_DIR)

os.makedirs(EMBEDDED_PROMPTS_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(IMAGES_DIR, exist_ok=True)
os.makedirs(FEATURES_DIR, exist_ok=True)

In [ ]:
!git checkout genetic-algorithm
!git pull

In [ ]:
import random

# Function to generate prompts
def generate_prompts(prompt_segments, num_prompts=6):
    # Select 6 random segments from the prompt_segments list
    selected_prompts = random.sample(prompt_segments, num_prompts)
    modifiers = [
        'beautiful', 'gorgeous', 'stunning', 'charming', 'captivating', 'breathtaking',
        'masterpiece', 'exquisite', 'magnificent', 'majestic', 'elegant', 'sublime',
        'ugly', 'hideous', 'grotesque', 'repulsive', 'disgusting', 'revolting',
        'futuristic', 'cyberpunk', 'hi-tech', 'advanced', 'innovative', 'modern',
        'fantasy', 'mythical', 'scifi', 'side scrolling', 'character', 'side scrolling',
        'white background', 'centered', 'full character', 'no background', 'not centered',
        'line drawing', 'sketch', 'black and white', 'colored', 'offset', 'video game']

    # Add modifiers to the selected prompts and return them in a list (PROMPT)
    return [f"{random.choice(modifiers)} {prompt}" for prompt in selected_prompts]

# List of prompt segments
prompt_segments = ['chibi', 'waifu', 'cyborg', 'dragon', 'android', 'nekomimi', 'mecha', 'kitsune', 'AI companion', 'furry detective', 'robot butler', 'futuristic steampunk', 'cybernetic implants',
                   'anthropomorphic AI', 'mechanical wizard', 'kemonomimi', 'android rebellion', 'magical robot pet', 'intergalactic furball', 'cyberpunk android', 'shapeshifting furry',
                   'mech pilot', 'furry time traveler']

# Generate 6 random prompts with modifiers (initial population)
PROMPT = generate_prompts(prompt_segments)

# Print the generated prompts
print(PROMPT)

In [ ]:
def embed_and_save_prompts(prompts: list):
    null_prompt = ""
    clip_text_embedder = CLIPTextEmbedder(device=get_device())
    clip_text_embedder.load_submodels()

    null_cond = clip_text_embedder(null_prompt)

    # Initialize list to store embeds
    embedded_prompts_list = []

    for prompt in prompts:
        # Embed each prompt individually
        print("Generating prompt ", prompt)
        embedded_prompt = clip_text_embedder(prompt)
        embedded_prompts_list.append(embedded_prompt)

    clip_text_embedder.to("cpu")
    del clip_text_embedder
    torch.cuda.empty_cache()
    get_memory_status()

    return (embedded_prompts_list, null_cond)

def normalized(a, axis=-1, order=2):
    import numpy as np

    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2 == 0] = 1
    return a / np.expand_dims(l2, axis)

# Generate embeddings for each prompt
embedding, null_prompt = embed_and_save_prompts(PROMPT)
num_images = len(embedding)

In [ ]:
# Load Stable Diffusion
sd = StableDiffusion(device=DEVICE, n_steps=N_STEPS)
sd.quick_initialize()
sd.model.load_unet()
sd.model.load_autoencoder().load_decoder()


# Load chadscore and clip
import clip

# Test calculate chadscore
chad = ChadScorePredictor(768)
chad.load_model("./input/model/aesthetic_scorer/sac+logos+ava1-l14-linearMSE.pth")
image_features_clip_model, preprocess = clip.load("ViT-L/14", device=get_device())  #RN50x64

In [ ]:
def generate_images_from_embeddings(embedded_prompts_array, null_prompt):
  # 'embedded_prompts_array' is the array of shape (i, 77, 768)
  image = sd.generate_images_from_embeddings(
      seed=SEED, embedded_prompt=embedded_prompts_array[i:i+1], null_prompt=null_prompt
  )
  return image


solution_reshaped = best_solution.reshape(1, 77, 768)
solution_reshaped = torch.tensor(solution_reshaped, dtype=torch.float32)

image = generate_images_from_embeddings(solution_reshaped, null_prompt)
    pil_image = to_pil(image[0])
    filename=f"{IMAGES_DIR}/{generation}.png"
    pil_image.show()
    pil_image.save(filename)

In [ ]:
import pygad

# Function to calculate the chad score for batch of images
def calculate_chad_score(ga_instance, solution, solution_idx):
  # Convert the solution back to the original shape (1, 77, 768)
  solution_reshaped = solution.reshape(1, 77, 768)

  # Convert the numpy array to a PyTorch tensor
  solution_reshaped = torch.tensor(solution_reshaped, dtype=torch.float32)

  # Copy the tensor to CUDA device if 'device' is 'cuda'
  if device == 'cuda':
    solution_reshaped = solution_reshaped.to(device)

  # Generate an image using the solution
  image = generate_images_from_embeddings(solution_reshaped, null_prompt)

  pil_image = to_pil(image[0])  # Convert to (height, width, channels)

  # If generation % 1 == 0 save
  if(ga_instance.generations_complete % 2 == 0):
    os.makedirs(f"{IMAGES_DIR}/{ga_instance.generations_complete}", exist_ok=True)
    filename=f"{IMAGES_DIR}/{ga_instance.generations_complete}/{idx}.png"
    pil_image.show()
    pil_image.save(filename)

  unsqueezed_image = preprocess(pil_image).unsqueeze(0).to(device)

  with torch.no_grad():
    image_features = image_features_clip_model.encode_image(unsqueezed_image)

    im_emb_arr = normalized(image_features.cpu().detach().numpy() )
    prediction = chad(torch.from_numpy(im_emb_arr).to(device).type(torch.cuda.FloatTensor))
    chad_score = prediction.item()
  return chad_score

# Define the GA loop function
def genetic_algorithm_loop(sd, embedded_prompts, null_prompt, generations=10, population_size=POPULATION_SIZE, mutation_rate=0.4, num_parents_mating=2):

    # Convert each tensor in the list to a 2D numpy array and store them in a new list
    embedded_prompts_list = []
    num_genes = embedded_prompts[0].shape[0] * embedded_prompts[0].shape[1]

    for tensor in embedded_prompts:
        tensor = tensor.cpu()
        list_tensor = tensor.squeeze().tolist()
        embedded_prompts_list.append(list_tensor)

    # Turn into a numpy array
    embedded_prompts_list = np.hstack(embedded_prompts_list)

    # Initialize the GA
    ga_instance = pygad.GA(num_generations=generations,
                              num_parents_mating=num_parents_mating,
                              fitness_func=calculate_chad_score,
                              num_genes=num_genes,
                              crossover_type="single_point",
                              initial_population=embedded_prompts_list,
                              mutation_percent_genes=mutation_rate*100)


    # Get the final best solution and images
    best_solution, best_solution_fitness = ga_instance.best_solution()
    best_solution = best_solution.reshape(1, 77, 768)  # Reshape the best solution to the correct shape
    final_image = sd.generate_images_from_embeddings(embedded_prompt=best_solution, null_prompt=null_prompt)

    # Save the final best solution images
    final_filename=f"{IMAGES_DIR}/final.png"
    pil_final = to_pil(final_image[0])
    pil_final.show()
    pil_final.save(final_filename)

    return best_solution

# Call the GA loop function with your initialized StableDiffusion model
best_solution = genetic_algorithm_loop(sd, embedding, null_prompt)

print("Best solution found!")

torch.save(embedded_prompts, join(EMBEDDED_PROMPTS_DIR, "embedded_final_solution.pt"))
print("Saving solution...")

In [ ]:
# Clean unused loaded models
del image_features_clip_model, sd